<a href="https://colab.research.google.com/github/cesaraugustt/br2025/blob/main/br2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [28]:
df = pd.read_csv('br2025.csv')
df.head()

,DataBruta,Data,DadosBruto,Home,xGH,GH,GA,Res,xGA,Away,oddHome,oddDraw,oddAway
0,"Dec 7, 4:00pm",07/12/2025,Fluminense 2.42 2 - 0 FT 0.79 Bahia,Fluminense,2.42,2,0,FT,0.79,Bahia,1.73,3.8,4.50
1,"Dec 7, 4:00pm",07/12/2025,Botafogo 2.00 4 - 2 FT 0.84 Fortaleza,Botafogo,2.00,4,2,FT,0.84,Fortaleza,1.84,3.7,3.40
2,"Dec 7, 4:00pm",07/12/2025,Corinthians 1.58 1 - 1 FT 0.63 Juventude,Corinthians,1.58,1,1,FT,0.63,Juventude,1.57,4.1,5.50
3,"Dec 7, 4:00pm",07/12/2025,Santos 1.58 3 - 0 FT 1.42 Cruzeiro,Santos,1.58,3,0,FT,1.42,Cruzeiro,1.36,4.8,8.25
4,"Dec 7, 4:00pm",07/12/2025,Atlético Mineiro 1.84 5 - 0 FT 0.89 Vasco da Gama,Atlético Mineiro,1.84,5,0,FT,0.89,Vasco da Gama,1.57,4.2,6.00


In [29]:
df = df[['Data', 'Home', 'Away', 'xGH', 'xGA', 'GH', 'GA', 'oddHome', 'oddDraw', 'oddAway']]
df.head()

,Data,Home,Away,xGH,xGA,GH,GA,oddHome,oddDraw,oddAway
0,07/12/2025,Fluminense,Bahia,2.42,0.79,2,0,1.73,3.8,4.50
1,07/12/2025,Botafogo,Fortaleza,2.00,0.84,4,2,1.84,3.7,3.40
2,07/12/2025,Corinthians,Juventude,1.58,0.63,1,1,1.57,4.1,5.50
3,07/12/2025,Santos,Cruzeiro,1.58,1.42,3,0,1.36,4.8,8.25
4,07/12/2025,Atlético Mineiro,Vasco da Gama,1.84,0.89,5,0,1.57,4.2,6.00


In [30]:
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
df.head()

,Data,Home,Away,xGH,xGA,GH,GA,oddHome,oddDraw,oddAway
0,2025-12-07,Fluminense,Bahia,2.42,0.79,2,0,1.73,3.8,4.50
1,2025-12-07,Botafogo,Fortaleza,2.00,0.84,4,2,1.84,3.7,3.40
2,2025-12-07,Corinthians,Juventude,1.58,0.63,1,1,1.57,4.1,5.50
3,2025-12-07,Santos,Cruzeiro,1.58,1.42,3,0,1.36,4.8,8.25
4,2025-12-07,Atlético Mineiro,Vasco da Gama,1.84,0.89,5,0,1.57,4.2,6.00


In [32]:
col_map = {'Data': 'Date', 'Home': 'HomeTeam', 'Away': 'AwayTeam', 'xGH': 'xHomeGoals', 'xGA': 'xAwayGoals', 'GH': 'HomeGoals', 'GA': 'AwayGoals'}

df = df.rename(columns=col_map)
df = df.sort_values('Date').reset_index(drop=True)
df.head()

,Date,HomeTeam,AwayTeam,xHomeGoals,xAwayGoals,HomeGoals,AwayGoals,oddHome,oddDraw,oddAway
0,2025-03-29,Juventude,Vitória,1.21,0.63,2,0,2.04,3.29,3.64
1,2025-03-29,Flamengo,Internacional,2.47,0.74,1,1,1.70,3.64,4.80
2,2025-03-29,São Paulo,Sport Recife,1.79,0.37,0,0,1.52,3.88,6.40
3,2025-03-29,Fortaleza,Fluminense,1.42,0.95,2,0,2.26,3.09,3.29
4,2025-03-29,Grêmio,Atlético Mineiro,1.68,0.68,2,1,2.14,3.32,3.33


In [33]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [36]:
goal_model_data = pd.concat([
  df[['HomeTeam', 'AwayTeam', 'HomeGoals']].assign(home=1).rename(
    columns={'HomeTeam': 'team', 'AwayTeam': 'oponent', 'HomeGoals': 'goals'}),
  df[['AwayTeam', 'HomeTeam', 'AwayGoals']].assign(home=0).rename(
    columns={'AwayTeam': 'team', 'HomeTeam': 'oponent', 'AwayGoals': 'goals'})
  ])

formula = 'goals ~ home + team + oponent'
model = smf.glm(formula=formula, data=goal_model_data, family=sm.families.Poisson()).fit()

In [37]:
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  goals   No. Observations:                  760
Model:                            GLM   Df Residuals:                      720
Model Family:                 Poisson   Df Model:                           39
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1035.3
Date:                Mon, 15 Dec 2025   Deviance:                       785.28
Time:                        22:52:28   Pearson chi2:                     662.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1833
Covariance Type:            nonrobust                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               